In [22]:
import cv2
import os

# video-path : Chemin de la vidéo à traiter
# output_folder : Dossier de destination
def video_to_frames(video_path, output_folder, frames_number):    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Chargement de la vidéo
    video_capture = cv2.VideoCapture(video_path)

    # Calcul du pas entre chaque frame pour obtenir le nombre requis
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_indices = [int(i * (total_frames - 1) / (frames_number - 1)) for i in range(frames_number)]

    # Lecture et extraction des frames
    count = 0  # Compteur pour nommer les frames
    extracted_frames = 0

    while True:
        success, image = video_capture.read()

        if not success:
            break

        if count in frames_indices:
            frame_path = os.path.join(output_folder, f"frame{extracted_frames}.jpg")
            cv2.imwrite(frame_path, image)
            extracted_frames += 1

        count += 1

    # Libération de la ressource vidéo
    video_capture.release()

In [23]:

output_folder1 = 'Personne_qui_marche/images'
video_path1 = 'Personne_qui_marche/Personne_qui_marche.mp4'

video_to_frames(video_path1, output_folder1, 10)

In [4]:
import ast
import os.path
from xml.dom import minidom

out_dir = './out'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

file = minidom.parse('annotations.xml')

images = file.getElementsByTagName('image')

for image in images:
    width = int(image.getAttribute('width'))
    height = int(image.getAttribute('height'))
    name = image.getAttribute('name')
    elem = image.getElementsByTagName('points')
    boxes = image.getElementsByTagName('box')  # Obtenez toutes les balises box pour cette image

    if len(boxes) > 0:  # Vérifiez s'il y a des balises box pour cette image
        bbox = boxes[0]  # Prenez la première balise box
        xtl = int(float(bbox.getAttribute('xtl')))
        ytl = int(float(bbox.getAttribute('ytl')))
        xbr = int(float(bbox.getAttribute('xbr')))
        ybr = int(float(bbox.getAttribute('ybr')))
        w = xbr - xtl
        h = ybr - ytl
        label_file = open(os.path.join(out_dir, name[:-4] + '.txt'), 'w')

        for e in elem:
            label_file.write('0 {} {} {} {} '.format(str((xtl + (w / 2)) / width), str((ytl + (h / 2)) / height),
                                                     str(w / width), str(h / height)))

            points = e.attributes['points']
            points = points.value.split(';')
            points_ = []
            for p in points:
                p = p.split(',')
                p1, p2 = p
                points_.append([int(float(p1)), int(float(p2))])
            for p_, p in enumerate(points_):
                label_file.write('{} {}'.format(p[0] / width, p[1] / height))
                if p_ < len(points_) - 1:
                    label_file.write(' ')
                else:
                    label_file.write('\n')

In [11]:
from ultralytics import YOLO


model = YOLO('yolov8n-pose.pt')  # load a pretrained model (recommended for training)

model.train(data='config.yaml', epochs=1, imgsz=640)


Ultralytics YOLOv8.0.215 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
engine\trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=config.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fa

train: Scanning C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\train.cache... 2 images,

train: WARNING ⚠️ C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\images\train\frame0.jpg: ignoring corrupt image/label: labels require 122 columns each
train: WARNING ⚠️ C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\images\train\frame1.jpg: ignoring corrupt image/label: labels require 122 columns each
WARNING ⚠️ No images found in C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.


ValueError: not enough values to unpack (expected 3, got 0)